In [ ]:
import h5py
import numpy as np

sensor_mask = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
                       45, 46, 47, 
                       120, 121, 122,
                       138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 
                       174, 175, 176, 177, 178, 179, 180, 181, 182,
                       198, 199, 200,
                       271, 272, 273, 274, 275, 276]

with h5py.File('./libribrain/COMPETITION_HOLDOUT/derivatives/serialised/sub-0_ses-2025_task-COMPETITION_HOLDOUT_run-1_proc-bads+headpos+sss+notch+bp+ds_meg.h5', 'r') as h5_file:
    # List all keys (data structure)
    print("Keys:", list(h5_file.keys()))
    scaling_factor = 1e-07
    # Select the dataset to extract (e.g. 'data')
    data = h5_file['data'][:].astype(np.float64)  # use [:] extract as ndarray
    masked_data = data[sensor_mask]
    normalized = masked_data / scaling_factor
# Save as .npy file to local disk
np.save('./libribrain/detection_holdout_48.npy', normalized.astype(np.float32))

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
# Assuming classification_model.py is in the same directory or accessible via PYTHONPATH
from seanet_style import SEANetTransformerClassifier, create_mask

# --- Dataset for Single File Evaluation with Overlap ---
class EvaluationDataset(Dataset):
    def __init__(self, input_filepath, chunk_size=2000, overlap=0):
        # Load the single input file using memory map
        self.input_mmap_full = np.load(input_filepath, mmap_mode='r')
        self.chunk_size = chunk_size
        self.overlap = overlap
        self.stride = chunk_size - overlap
        if self.stride <= 0:
            raise ValueError("Overlap must be less than chunk_size.")

        self.data_chunks = []
        self.chunk_start_indices = [] # To keep track of original positions

        total_input_len = self.input_mmap_full.shape[1]

        for start_idx in range(0, total_input_len, self.stride):
            end_idx = min(start_idx + chunk_size, total_input_len)
            
            # Adjust start_idx for the last chunk to ensure it is full-sized if possible
            if end_idx - start_idx < chunk_size and end_idx == total_input_len:
                start_idx = max(0, total_input_len - chunk_size)

            input_chunk = torch.from_numpy(self.input_mmap_full[:, start_idx:end_idx]).float()
            
            # Pad the last chunk if it's smaller than chunk_size
            current_chunk_len = input_chunk.shape[1]
            if current_chunk_len < chunk_size:
                padding_needed = chunk_size - current_chunk_len
                # Pad only the sequence dimension
                input_chunk = F.pad(input_chunk, (0, padding_needed))
            
            self.data_chunks.append((input_chunk, input_chunk.shape[1]))
            self.chunk_start_indices.append(start_idx) # Store original start index

    def __len__(self):
        return len(self.data_chunks)

    def __getitem__(self, idx):
        input_chunk, original_len = self.data_chunks[idx]
        start_idx = self.chunk_start_indices[idx]
        return input_chunk, original_len, start_idx

# --- Collate Function (adapted for new dataset output) ---
def collate_fn(batch):
    inputs, input_lengths_orig, start_indices = zip(*batch)
    
    input_lengths = list(input_lengths_orig)
    
    max_input_len = max(input_lengths)
    # No padding needed here if EvaluationDataset already pads to chunk_size
    # However, if it's dynamic chunking, this still applies.
    # Given the updated EvaluationDataset, input_chunk will always be chunk_size
    # so max_input_len will be chunk_size.
    padded_inputs = torch.stack([x for x in inputs])
    
    return padded_inputs, input_lengths, list(start_indices)

# --- Helper function to calculate Classification Metrics (F1-score) ---
def calculate_classification_metrics(all_predictions_logits, all_targets, threshold=0.5):
    all_predictions_probs = torch.sigmoid(torch.tensor(all_predictions_logits)).numpy()
    all_predictions_binary = (all_predictions_probs >= threshold).astype(int)
    all_targets_np = np.array(all_targets)

    all_predictions_binary_flat = all_predictions_binary.flatten()
    all_targets_np_flat = all_targets_np.flatten()

    f1 = f1_score(all_targets_np_flat, all_predictions_binary_flat, zero_division=0)
    precision = precision_score(all_targets_np_flat, all_predictions_binary_flat, zero_division=0)
    recall = recall_score(all_targets_np_flat, all_predictions_binary_flat, zero_division=0)
    
    if len(np.unique(all_targets_np_flat)) > 1:
        roc_auc = roc_auc_score(all_targets_np_flat, all_predictions_probs.flatten())
    else:
        roc_auc = float('nan')

    return f1, precision, recall, roc_auc

def infer_single_file(model, data_file_path, output_dir, device, 
                      chunk_size, overlap_size, batch_size, detection_threshold):
    
    base_filename = os.path.basename(data_file_path)
    
    # Load the entire numpy array
    full_data = np.load(data_file_path)
    # Ensure it's (channels, time) or (time, channels)
    # Assuming full_data is (time_steps, channels) or (channels, time_steps)
    # If it's (time_steps, channels), transpose it to (channels, time_steps)
    if full_data.shape[0] == 48 and full_data.ndim == 2: # Already (channels, time)
        original_length = full_data.shape[1]
    elif full_data.shape[1] == 48 and full_data.ndim == 2: # (time, channels)
        full_data = full_data.T # Transpose to (channels, time)
        original_length = full_data.shape[1]
    elif full_data.ndim == 1: # (time,)
        raise ValueError("Input data cannot be 1D. Expected (channels, time) or (time, channels).")
    else: # Already 3D, e.g., (1, channels, time) or (batch, channels, time)
        # Assuming it's a single long sequence, taking the first one if batch exists
        if full_data.ndim == 3:
            full_data = full_data[0] # Take the first sequence if it's (1, C, T) or (B, C, T)
            if full_data.shape[0] != 48: # Transpose if C is not first
                full_data = full_data.T
        if full_data.shape[0] != 48:
             raise ValueError(f"Input data has {full_data.shape[0]} channels. Expected 23 after potential transpose.")
        original_length = full_data.shape[1]

    # Create chunks for inference
    chunks = []
    start_indices = []
    
    step_size = chunk_size - overlap_size
    if step_size <= 0:
        raise ValueError("Step size (chunk_size - overlap_size) must be positive.")

    for i in range(0, original_length, step_size):
        start = i
        end = min(i + chunk_size, original_length)
        
        # Pad if the last chunk is smaller than chunk_size
        current_chunk = full_data[:, start:end]
        current_length = current_chunk.shape[1]

        if current_length < chunk_size:
            # Pad with zeros to chunk_size
            padding_needed = chunk_size - current_length
            padded_chunk = np.pad(current_chunk, ((0, 0), (0, padding_needed)), mode='constant')
            chunks.append(padded_chunk)
            start_indices.append(start)
            # This is the last chunk, so break
            break 
        else:
            chunks.append(current_chunk)
            start_indices.append(start)

    # Convert chunks to tensor dataset
    class InferenceDataset(torch.utils.data.Dataset):
        def __init__(self, chunks, start_indices):
            self.chunks = chunks
            self.start_indices = start_indices
        
        def __len__(self):
            return len(self.chunks)
        
        def __getitem__(self, idx):
            chunk = torch.from_numpy(self.chunks[idx]).float()
            # input_length for the model is the actual length of the data in the chunk (before padding)
            # This is critical if the model uses original_lengths for internal calculations.
            # When padding, input_length should be the length *before* padding.
            # However, your model now trims/pads output, so the actual length of the tensor passed is fine.
            # Let's just pass the chunk_size as input_length for simplicity with the padding.
            return chunk, torch.tensor(chunk.shape[1], dtype=torch.long), torch.tensor(self.start_indices[idx], dtype=torch.long)

    inference_dataset = InferenceDataset(chunks, start_indices)
    evaluation_dataloader = torch.utils.data.DataLoader(
        inference_dataset, batch_size=batch_size, shuffle=False, num_workers=0
    )

    fused_predictions_logits = np.zeros(original_length, dtype=np.float32)
    prediction_counts = np.zeros(original_length, dtype=np.int32)

    model.eval() # Set model to evaluation mode
    with torch.no_grad():
        progress_bar = tqdm(evaluation_dataloader, desc=f"Inferring {base_filename} with overlap")
        for batch_idx, batch in enumerate(progress_bar):
            inputs_batch, input_lengths_batch, start_indices_batch = batch
            inputs_batch = inputs_batch.to(device)

            # Convert input_lengths_batch to a list of Python integers if your model expects it
            # The model's forward method expects `original_lengths: tp.List[int]`
            input_lengths_list = input_lengths_batch.tolist()

            predictions_logits_batch = model(inputs_batch, original_lengths=input_lengths_list).cpu().numpy() # (batch_size, 1, chunk_size)

            for i, start_idx in enumerate(start_indices_batch):
                current_input_length = input_lengths_list[i] # Use the actual length passed to model
                
                # predictions_logits_batch[i, 0, ...] should be of length `current_input_length` (1200)
                chunk_prediction_logits = predictions_logits_batch[i, 0, :current_input_length]
                
                end_idx = min(original_length, start_idx.item() + current_input_length) # .item() for scalar tensor
                effective_chunk_len = end_idx - start_idx.item()
                

                fused_predictions_logits[start_idx.item():end_idx] += chunk_prediction_logits[:effective_chunk_len]
                prediction_counts[start_idx.item():end_idx] += 1

    # Avoid division by zero for positions not covered by any chunk (though unlikely with proper overlap)
    prediction_counts[prediction_counts == 0] = 1

    # Final averaging
    full_predictions_logits = fused_predictions_logits / prediction_counts
    
    # Convert logits to probabilities
    full_predictions_probs = torch.sigmoid(torch.from_numpy(full_predictions_logits)).numpy()

    # Apply threshold for binary classification
    binary_predictions = (full_predictions_probs >= detection_threshold).astype(np.int32)

    # Save predictions
    output_probs_filepath = os.path.join(output_dir, f"prediction_probs_{base_filename}")
    np.save(output_probs_filepath, full_predictions_probs)
    output_binary_filepath = os.path.join(output_dir, f"prediction_binary_{base_filename}")
    np.save(output_binary_filepath, binary_predictions)

    print(f"Probabilities for {base_filename} saved to {output_probs_filepath}")
    print(f"Binary predictions for {base_filename} saved to {output_binary_filepath}")

    print(f"\nInference complete for {data_file_path}.")
    print(f"Shape of predicted binary output: {binary_predictions.shape}")
    print(f"First 20 binary predictions: {binary_predictions[:20].tolist()}")
    print(f"First 20 probabilities: {full_predictions_probs[:20].tolist()}")
    
    return full_predictions_probs, binary_predictions


if __name__ == "__main__":
    # Configuration
    model_path = "./libribrain/speech_detection_final.pth"  # Path to your trained model
    input_file_for_evaluation = './libribrain/detection_holdout_48.npy'  # Replace with the actual input file path
    output_prediction_dir = "./libribrain/predictions"  # Directory to save the output prediction

    chunk_size_for_inference = 1000  # Must match the chunking used during validation/inference
    overlap_for_inference = 950     # New: Overlap size. E.g., 200 for 50% overlap if chunk_size is 400.
    
    num_input_channels = 48  # Number of channels in your input data
    num_output_channels = 1   # Model outputs 1 channel for binary classification
    detection_threshold = 0.5

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Create output directory if it doesn't exist
    os.makedirs(output_prediction_dir, exist_ok=True)
    
    # Model parameters for SEANetTransformerClassifier
    model_params = {
        'input_channels': 48,
        'sampling_rate': 250,
        'encoder_dimension': 16, # Feature dimension after encoder
        'encoder_n_filters': 8,  # Base number of filters
        'encoder_ratios': [5, 5, 2], # Encoder downsamples by these ratios (reversed in encoder). Total downsample = 5*5*2 = 50.
        'transformer_n_heads': 4, # Reduced heads for smaller dimension
        'transformer_n_layers': 1, # Reduced layers for faster iteration
        'transformer_dim_feedforward': 64, # Smaller feedforward dim
        'transformer_dropout': 0.1,
        'decoder_out_channels': 1, # Binary classification logits
            
        # SEANet-style parameters for Encoder/Decoder blocks
        'activation': 'GELU', 
        'activation_params': {}, 
        'norm': 'InstanceNorm1d', # Using LayerNorm for simplicity
        'norm_params': {}, 
        'n_residual_layers': 1, # Number of residual layers per stage
        'kernel_size': 7, 
        'last_kernel_size': 7, 
        'residual_kernel_size': 3, 
        'dilation_base': 2, 
        'compress': 2, 
        'true_skip': False # Use conv shortcut in resnet block
    }
    model = SEANetTransformerClassifier(**model_params).to(device).float()

    # Load trained model weights
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path, map_location=device))
        print(f"Loaded model weights from {model_path}")
    else:
        print(f"Error: Model weights not found at {model_path}. Please train the model first or provide the correct path.")
        exit()

    # Perform inference on the single file with overlap
    print(f"Starting inference for: {input_file_for_evaluation} with chunk_size={chunk_size_for_inference}, overlap={overlap_for_inference}")
    predicted_binary_output, predicted_probabilities = infer_single_file(
        model, 
        input_file_for_evaluation, 
        output_prediction_dir,
        device, 
        chunk_size_for_inference, 
        overlap_for_inference, # Pass the overlap value
        batch_size = 128,
        detection_threshold = 0.5
    )
    print(f"\nInference complete for {input_file_for_evaluation}.")
    print(f"Shape of predicted binary output: {predicted_binary_output.shape}")


In [ ]:
from pnpl.datasets import LibriBrainCompetitionHoldout
from torch.utils.data import DataLoader
import torch
import numpy as np
# First, instantiate the Competition Holdout dataset
speech_holdout_dataset = LibriBrainCompetitionHoldout(
    data_path="./libribrain",  # Same as in the other LibriBrain dataset - this is where we'll store the data
    tmax=0.8,             # Also identical to the other datasets - how many samples to return/group together
    task="speech"         # "speech" or "phoneme" ("phoneme" is not supported until Phoneme track launch)
)

predictions = np.load("./libribrain/predictions/prediction_probs_detection_holdout_48.npy").reshape(-1, 1) 

print(predictions.shape)

tensor=torch.from_numpy(predictions)

speech_holdout_dataset.generate_submission_in_csv(
    tensor,
    "./libribrain/holdout_speech_predictions.csv"
)

